In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import concurrent.futures
import time
import threading

# Replace with your Spotify Developer credentials
SPOTIPY_CLIENT_ID = '6692ee51d1fe43f1837fcd26ee450018'
SPOTIPY_CLIENT_SECRET = '3d07720ecc6944aebe7a8b4da9ea876a'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

data = pd.read_csv("unique_songs.csv")

# Set up authentication
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope=scope))

# Memo cache dictionary
memo_cache = {}

# Lock for thread-safe memo cache updates
lock = threading.Lock()

# Counter for consecutive 429 responses
consecutive_429 = 0

# Function to search for song and get the song ID


def get_song_id(row):
    global consecutive_429
    artist = row['Artist']
    track = row['Song']

    # Check if the song ID is already in the cache
    cache_key = f"{artist}:{track}"
    with lock:
        if cache_key in memo_cache:
            return cache_key, memo_cache[cache_key]

    # Replace spaces with %20
    formatted_artist = artist.replace(" ", "%20")
    formatted_track = track.replace(" ", "%20")
    query = f"remaster%20track:{formatted_track}%20artist:{formatted_artist}"

    try:
        result = sp.search(q=query, type='track', limit=1, market='CA')
        if result['tracks']['items']:
            song_id = result['tracks']['items'][0]['id']
            with lock:
                # Store the song ID in the cache
                memo_cache[cache_key] = song_id
            consecutive_429 = 0  # Reset the counter on successful response
            return cache_key, song_id
        else:
            return cache_key, None
    except spotipy.exceptions.SpotifyException as e:
        if e.http_status == 429:
            consecutive_429 += 1
            if consecutive_429 >= 50:
                raise RuntimeError(
                    "Too many consecutive 429 errors. Exiting...")
        else:
            consecutive_429 = 0
        return cache_key, None

# Function to process rows with rate limiting


def process_row_with_rate_limit(row):
    while True:
        try:
            cache_key, song_id = get_song_id(row)
            time.sleep(0.5)  # 20 requests per second
            return cache_key, song_id
        except RuntimeError as e:
            # Stop sending requests and save progress
            with lock:
                data.to_csv('songs_with_ids_partial.csv', index=False)
            raise e


count = 1
# Use ThreadPoolExecutor to send API requests concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(
        process_row_with_rate_limit, row): row for index, row in data.iterrows()}
    for future in concurrent.futures.as_completed(futures):
        cache_key, song_id = future.result()
        with lock:
            memo_cache[cache_key] = song_id
            data.loc[data['Artist'] + ":" + data['Song']
                     == cache_key, 'Song ID'] = song_id
            print(f"Processed: {count} / {len(data)}")
            count += 1

print(data)


Processed: 1 / 8712
Processed: 2 / 8712
Processed: 3 / 8712
Processed: 4 / 8712
Processed: 5 / 8712
Processed: 6 / 8712
Processed: 7 / 8712
Processed: 8 / 8712
Processed: 9 / 8712
Processed: 10 / 8712
Processed: 11 / 8712
Processed: 12 / 8712
Processed: 13 / 8712
Processed: 14 / 8712
Processed: 15 / 8712
Processed: 16 / 8712
Processed: 17 / 8712
Processed: 18 / 8712
Processed: 19 / 8712
Processed: 20 / 8712
Processed: 21 / 8712
Processed: 22 / 8712
Processed: 23 / 8712
Processed: 24 / 8712
Processed: 25 / 8712
Processed: 26 / 8712
Processed: 27 / 8712
Processed: 28 / 8712
Processed: 29 / 8712
Processed: 30 / 8712
Processed: 31 / 8712
Processed: 32 / 8712
Processed: 33 / 8712
Processed: 34 / 8712
Processed: 35 / 8712
Processed: 36 / 8712
Processed: 37 / 8712
Processed: 38 / 8712
Processed: 39 / 8712
Processed: 40 / 8712
Processed: 41 / 8712
Processed: 42 / 8712
Processed: 43 / 8712
Processed: 44 / 8712
Processed: 45 / 8712
Processed: 46 / 8712
Processed: 47 / 8712
Processed: 48 / 8712
P

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'remaster%20track:Lieder%20ohne%20Worte%20(Songs%20without%20Words)%20Book%201%20Op.%2019b:%20Lied%20ohne%20Worte%20(Song%20without%20Words)%20No.%206%20in%20G%20minor%20Op.%2019%20No.%206%20Venezianisches%20Gondellied%20(Venetian%20Gondola%20Song)%20artist:Felix%20Mendelssohn', 'limit': 1, 'offset': 0, 'type': 'track', 'market': 'CA'} returned 400 due to Bad request.


Processed: 301 / 8712
Processed: 302 / 8712
Processed: 303 / 8712
Processed: 304 / 8712
Processed: 305 / 8712
Processed: 306 / 8712
Processed: 307 / 8712
Processed: 308 / 8712
Processed: 309 / 8712
Processed: 310 / 8712
Processed: 311 / 8712
Processed: 312 / 8712
Processed: 313 / 8712
Processed: 314 / 8712
Processed: 315 / 8712
Processed: 316 / 8712
Processed: 317 / 8712
Processed: 318 / 8712
Processed: 319 / 8712
Processed: 320 / 8712
Processed: 321 / 8712
Processed: 322 / 8712
Processed: 323 / 8712
Processed: 324 / 8712
Processed: 325 / 8712
Processed: 326 / 8712
Processed: 327 / 8712
Processed: 328 / 8712
Processed: 329 / 8712
Processed: 330 / 8712
Processed: 331 / 8712
Processed: 332 / 8712
Processed: 333 / 8712
Processed: 334 / 8712
Processed: 335 / 8712
Processed: 336 / 8712
Processed: 337 / 8712
Processed: 338 / 8712
Processed: 339 / 8712
Processed: 340 / 8712
Processed: 341 / 8712
Processed: 342 / 8712
Processed: 343 / 8712
Processed: 344 / 8712
Processed: 345 / 8712
Processed:

Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached


Processed: 362 / 8712
Processed: 363 / 8712
Processed: 364 / 8712
Processed: 365 / 8712
Processed: 366 / 8712
Processed: 367 / 8712
Processed: 368 / 8712
Processed: 369 / 8712
Processed: 370 / 8712
Processed: 371 / 8712
Processed: 372 / 8712
Processed: 373 / 8712
Processed: 374 / 8712
Processed: 375 / 8712
Processed: 376 / 8712
Processed: 377 / 8712
Processed: 378 / 8712
Processed: 379 / 8712
Processed: 380 / 8712
Processed: 381 / 8712
Processed: 382 / 8712
Processed: 383 / 8712
Processed: 384 / 8712
Processed: 385 / 8712
Processed: 386 / 8712
Processed: 387 / 8712
Processed: 388 / 8712
Processed: 389 / 8712
Processed: 390 / 8712
Processed: 391 / 8712
Processed: 392 / 8712
Processed: 393 / 8712
Processed: 394 / 8712
Processed: 395 / 8712
Processed: 396 / 8712
Processed: 397 / 8712
Processed: 398 / 8712
Processed: 399 / 8712
Processed: 400 / 8712
Processed: 401 / 8712
Processed: 402 / 8712
Processed: 403 / 8712
Processed: 404 / 8712
Processed: 405 / 8712
Processed: 406 / 8712
Processed:

...

Processed: 7839 / 8712
Processed: 7840 / 8712
Processed: 7841 / 8712
Processed: 7842 / 8712
Processed: 7843 / 8712
Processed: 7844 / 8712
Processed: 7845 / 8712
Processed: 7846 / 8712
Processed: 7847 / 8712
Processed: 7848 / 8712
Processed: 7849 / 8712
Processed: 7850 / 8712
Processed: 7851 / 8712
Processed: 7852 / 8712
Processed: 7853 / 8712
Processed: 7854 / 8712
Processed: 7855 / 8712
Processed: 7856 / 8712
Processed: 7857 / 8712
Processed: 7858 / 8712
Processed: 7859 / 8712
Processed: 7860 / 8712
Processed: 7861 / 8712
Processed: 7862 / 8712
Processed: 7863 / 8712
Processed: 7864 / 8712
Processed: 7865 / 8712
Processed: 7866 / 8712
Processed: 7867 / 8712
Processed: 7868 / 8712
Processed: 7869 / 8712
Processed: 7870 / 8712
Processed: 7871 / 8712
Processed: 7872 / 8712
Processed: 7873 / 8712
Processed: 7874 / 8712
Processed: 7875 / 8712
Processed: 7876 / 8712
Processed: 7877 / 8712
Processed: 7878 / 8712
Processed: 7879 / 8712
Processed: 7880 / 8712
Processed: 7881 / 8712
Processed: 

Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached


Processed: 8218 / 8712
Processed: 8219 / 8712
Processed: 8220 / 8712
Processed: 8221 / 8712
Processed: 8222 / 8712
Processed: 8223 / 8712
Processed: 8224 / 8712
Processed: 8225 / 8712
Processed: 8226 / 8712
Processed: 8227 / 8712
Processed: 8228 / 8712
Processed: 8229 / 8712
Processed: 8230 / 8712
Processed: 8231 / 8712
Processed: 8232 / 8712
Processed: 8233 / 8712
Processed: 8234 / 8712
Processed: 8235 / 8712
Processed: 8236 / 8712
Processed: 8237 / 8712
Processed: 8238 / 8712
Processed: 8239 / 8712
Processed: 8240 / 8712
Processed: 8241 / 8712
Processed: 8242 / 8712
Processed: 8243 / 8712
Processed: 8244 / 8712
Processed: 8245 / 8712
Processed: 8246 / 8712
Processed: 8247 / 8712
Processed: 8248 / 8712
Processed: 8249 / 8712
Processed: 8250 / 8712
Processed: 8251 / 8712
Processed: 8252 / 8712
Processed: 8253 / 8712
Processed: 8254 / 8712
Processed: 8255 / 8712
Processed: 8256 / 8712
Processed: 8257 / 8712
Processed: 8258 / 8712
Processed: 8259 / 8712
Processed: 8260 / 8712
Processed: 

Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached


Processed: 8343 / 8712
Processed: 8344 / 8712
Processed: 8345 / 8712
Processed: 8346 / 8712
Processed: 8347 / 8712
Processed: 8348 / 8712
Processed: 8349 / 8712
Processed: 8350 / 8712
Processed: 8351 / 8712
Processed: 8352 / 8712
Processed: 8353 / 8712
Processed: 8354 / 8712
Processed: 8355 / 8712
Processed: 8356 / 8712
Processed: 8357 / 8712
Processed: 8358 / 8712
Processed: 8359 / 8712
Processed: 8360 / 8712
Processed: 8361 / 8712
Processed: 8362 / 8712
Processed: 8363 / 8712
Processed: 8364 / 8712
Processed: 8365 / 8712
Processed: 8366 / 8712
Processed: 8367 / 8712
Processed: 8368 / 8712
Processed: 8369 / 8712
Processed: 8370 / 8712
Processed: 8371 / 8712
Processed: 8372 / 8712
Processed: 8373 / 8712
Processed: 8374 / 8712
Processed: 8375 / 8712
Processed: 8376 / 8712
Processed: 8377 / 8712
Processed: 8378 / 8712
Processed: 8379 / 8712
Processed: 8380 / 8712
Processed: 8381 / 8712
Processed: 8382 / 8712
Processed: 8383 / 8712
Processed: 8384 / 8712
Processed: 8385 / 8712
Processed: 

In [2]:
data.head()

,Artist,Album,Song,Date Played,Song ID
0,Drake,Views,One Dance,2024-07-07 23:10,1zi7xx7UVEFkmKfv06H8x0
1,A$AP Rocky,TESTING,Praise the Lord (Da Shine) (feat. Skepta),2024-07-07 23:06,7ycWLEP1GsNjVvcjawXz3z
2,Travis Scott,ASTROWORLD,SICKO MODE,2024-07-07 23:01,2xLMifQCjDGFmkHkpNLD9h
3,Kanye West,Donda,Hurricane,2024-07-07 22:57,6Hfu9sc7jvv6coyy2LlzBF
4,Gunna,WUNNA,DOLLAZ ON MY HEAD (feat. Young Thug),2024-07-07 22:54,3nS9a01VvXHQriLqJYwRqG


In [3]:
data.to_csv("songs_with_ids.csv", index=False)

In [2]:
import pandas as pd
data = pd.read_csv("songs_with_ids.csv")

missing_ids = data[data['Song ID'].isnull()]
print(missing_ids)

                 Artist                                    Album  \
327   Felix Mendelssohn  MENDELSSOHN: Songs without Words Vol. 2   
361             G Mills                              Lazy Sunday   
362         Dr. Dundiff                              Lazy Sunday   
363             Thaehan                              Lazy Sunday   
364      Blue Wednesday                              Lazy Sunday   
...                 ...                                      ...   
8355              elsei                              Wait For Me   
8356         Jean Thiel                             Ocean Voices   
8357      Burrito Brown              Bluewerks Vol. 3: Heat Wave   
8362         Sleep Guru                                 Midnight   
8363              Leave                            Hot Chocolate   

                                                   Song       Date Played  \
327   Lieder ohne Worte (Songs without Words) Book 1...   2024-06-25 4:06   
361                          

In [9]:
missing_ids = pd.read_csv("missing_ids.csv")
missing_ids.head()

,Artist,Album,Song,Date Played,Song ID
0,Felix Mendelssohn,MENDELSSOHN: Songs without Words Vol. 2,Lieder ohne Worte (Songs without Words) Book 1...,2024-06-25 4:06,NaN
1,Kupla,Mirage,We Were Young,2024-06-19 20:47,NaN
2,Kupla,Taiga Native,Can You See the Rhino in the Clouds?,2024-02-15 21:06,NaN
3,eugenio izzi,Feel Free To Imagine,The Fairie's City,2024-01-27 6:20,NaN
4,Chau Sara,Sleepless Night,Flying Doodles,2024-01-06 22:18,NaN


In [11]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Replace with your Spotify Developer credentials
SPOTIPY_CLIENT_ID = '6692ee51d1fe43f1837fcd26ee450018'
SPOTIPY_CLIENT_SECRET = '3d07720ecc6944aebe7a8b4da9ea876a'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

data = missing_ids
# Set up authentication
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope=scope))

# Memo cache dictionary
memo_cache = {}

# Function to search for song and get the song ID
def get_song_id(artist, track):
    # Check if the song ID is already in the cache
    try:
        cache_key = f"{artist}:{track}"
        if cache_key in memo_cache:
            return memo_cache[cache_key]

        # Replace spaces with %20
        formatted_artist = artist.replace(" ", "%20")
        formatted_track = track.replace(" ", "%20")
        query = f"remaster%20track:{formatted_track}%20artist:{formatted_artist}"
        print(query)
        result = sp.search(q=query, type='track', limit=1, market='CA')
        print(result)
        if result['tracks']['items']:
            song_id = result['tracks']['items'][0]['id']
            memo_cache[cache_key] = song_id  # Store the song ID in the cache
            return song_id
        else:
            return None
    except spotipy.exceptions.SpotifyException as e:
        print(e)
        return None

# Apply the function to the DataFrame
data['Song ID'] = data.apply(lambda row: get_song_id(row['Artist'], row['Song']), axis=1)

print(data)

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'remaster%20track:Lieder%20ohne%20Worte%20(Songs%20without%20Words)%20Book%201%20Op.%2019b:%20Lied%20ohne%20Worte%20(Song%20without%20Words)%20No.%206%20in%20G%20minor%20Op.%2019%20No.%206%20Venezianisches%20Gondellied%20(Venetian%20Gondola%20Song)%20artist:Felix%20Mendelssohn', 'limit': 1, 'offset': 0, 'type': 'track', 'market': 'CA'} returned 400 due to Bad request.


remaster%20track:Lieder%20ohne%20Worte%20(Songs%20without%20Words)%20Book%201%20Op.%2019b:%20Lied%20ohne%20Worte%20(Song%20without%20Words)%20No.%206%20in%20G%20minor%20Op.%2019%20No.%206%20Venezianisches%20Gondellied%20(Venetian%20Gondola%20Song)%20artist:Felix%20Mendelssohn
http status: 400, code:-1 - https://api.spotify.com/v1/search?q=remaster%2520track%3ALieder%2520ohne%2520Worte%2520%28Songs%2520without%2520Words%29%2520Book%25201%2520Op.%252019b%3A%2520Lied%2520ohne%2520Worte%2520%28Song%2520without%2520Words%29%2520No.%25206%2520in%2520G%2520minor%2520Op.%252019%2520No.%25206%2520Venezianisches%2520Gondellied%2520%28Venetian%2520Gondola%2520Song%29%2520artist%3AFelix%2520Mendelssohn&limit=1&offset=0&type=track&market=CA:
 Bad request., reason: None
remaster%20track:We%20Were%20Young%20artist:Kupla
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=remaster%2520track%3AWe%2520Were%2520Young%2520artist%3AKupla&type=track&market=CA&offset=0&limit=1', 'items': [], 'limit'

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'remaster%20track:The%20Sleeping%20Beauty%20Op.%2066%20:%20Prologue%20No.3%20Pas%20de%20six%20Variation%20I.%20Candile%20(Fairy%20of%20the%20Crystal%20Fountain)%20artist:Saint%20Petersburg%20Radio%20and%20TV%20Symphony%20Orchestra%20Stanislav%20Gorkovenko', 'limit': 1, 'offset': 0, 'type': 'track', 'market': 'CA'} returned 400 due to Bad request.


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=remaster%2520track%3ANumber%25207%2520%28Outro%29%2520artist%3APapii%2520Don%2520777&type=track&market=CA&offset=0&limit=1', 'items': [], 'limit': 1, 'next': None, 'offset': 0, 'previous': None, 'total': 0}}
remaster%20track:Merikarp%20artist:Fabien%20Tamm
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=remaster%2520track%3AMerikarp%2520artist%3AFabien%2520Tamm&type=track&market=CA&offset=0&limit=1', 'items': [], 'limit': 1, 'next': None, 'offset': 0, 'previous': None, 'total': 0}}
remaster%20track:The%20Sleeping%20Beauty%20Op.%2066%20:%20Prologue%20No.3%20Pas%20de%20six%20Variation%20I.%20Candile%20(Fairy%20of%20the%20Crystal%20Fountain)%20artist:Saint%20Petersburg%20Radio%20and%20TV%20Symphony%20Orchestra%20Stanislav%20Gorkovenko
http status: 400, code:-1 - https://api.spotify.com/v1/search?q=remaster%2520track%3AThe%2520Sleeping%2520Beauty%2520Op.%252066%2520%3A%2520Prologue%2520No.3%2520Pas%2520de%2520six%252

In [3]:
import pandas as pd

# Read the songs with IDs CSV
songs_with_ids = pd.read_csv("songs_with_ids.csv")

# Read the missing IDs CSV
missing_ids = pd.read_csv("missing_ids.csv")

# Merge the dataframes based on the common columns
merged_data = pd.merge(songs_with_ids, missing_ids, on=["Artist", "Album", "Song", "Date Played"], how="left")

# Update the Song ID column with the IDs from the missing IDs dataframe
merged_data["Song ID_x"].fillna(merged_data["Song ID_y"], inplace=True)

# Drop the unnecessary columns
merged_data.drop(columns=["Song ID_y"], inplace=True)
merged_data.rename(columns={"Song ID_x": "Song ID"}, inplace=True)

new = merged_data[merged_data['Song ID'].isnull()]

merged_data.to_csv("final_songs_with_ids.csv", index=False)

C:\Users\allan\AppData\Local\Temp\ipykernel_38056\201309465.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data["Song ID_x"].fillna(merged_data["Song ID_y"], inplace=True)


In [4]:
import pandas as pd

# Read the final_songs_with_ids.csv file
final_songs_with_ids = pd.read_csv("final_songs_with_ids.csv")

# Read the arandomguy608.csv file
arandomguy608 = pd.read_csv("arandomguy608.csv")

# Merge the dataframes based on the common columns (Artist and Song)
merged_data = pd.merge(arandomguy608, final_songs_with_ids, on=["Artist", "Song"], how="left")

# Save the updated dataframe to a new CSV file
merged_data.to_csv("arandomguy608_with_ids.csv", index=False)